In [206]:
import sys, os
import glob
from pprint import pprint
import csv
from pathlib import Path
import numpy as np
import shutil

In [207]:
DATA = Path('/data-nas/brains/IXI_Dataset/interim_copy_test')

In [208]:
studies_path = sorted(glob.glob(str(DATA)+'/*'))

In [209]:
from utils import write_nifti, load_nifti

In [210]:
def unique_values(studies_path):
 
    count_values = {}
    for path in studies_path:
        study_id = (path.split("/")[-1])
        imgfn = (path + "/"+ study_id +"_seg.nii.gz")
        
        img, rnx, rny, rnz, data, affine = load_nifti(imgfn)
        
        np_arr = np.unique(data) #unqiue values in a matrix 
        for i in np_arr: 
            if i in count_values: count_values[i] += 1 
            else: count_values[i] = 1
                
    return count_values

In [211]:
dic_classes = unique_values(studies_path)

In [212]:
num_images = dic_classes[0] #klasse 0 vil vaere i alle bildene, altså 577 for IXI 
dc = {}
i = 0.
#hvis ikke klassen samsvarer med num_images, så settes den til 0 
for key, value in dic_classes.items():
    if value != num_images: 
        dc[key] = 0.
    else: 
        dc[key] = i 
        i +=1

In [213]:
dc

{0.0: 0.0,
 2.0: 1.0,
 3.0: 2.0,
 4.0: 3.0,
 5.0: 4.0,
 7.0: 5.0,
 8.0: 6.0,
 10.0: 7.0,
 11.0: 8.0,
 12.0: 9.0,
 13.0: 10.0,
 14.0: 11.0,
 15.0: 12.0,
 16.0: 13.0,
 17.0: 14.0,
 18.0: 15.0,
 24.0: 16.0,
 26.0: 17.0,
 28.0: 18.0,
 30.0: 0.0,
 31.0: 19.0,
 41.0: 20.0,
 42.0: 21.0,
 43.0: 22.0,
 44.0: 23.0,
 46.0: 24.0,
 47.0: 25.0,
 49.0: 26.0,
 50.0: 27.0,
 51.0: 28.0,
 52.0: 29.0,
 53.0: 30.0,
 54.0: 31.0,
 58.0: 32.0,
 60.0: 33.0,
 62.0: 0.0,
 63.0: 34.0,
 77.0: 35.0,
 85.0: 36.0,
 251.0: 37.0,
 252.0: 38.0,
 253.0: 39.0,
 254.0: 40.0,
 255.0: 41.0,
 72.0: 0.0,
 80.0: 0.0}

In [214]:
#sett alle klassene som ikke er med i listen til 0 
def classes_to_keep(dic, keep_classes, binary = False): 
    mod_dic = {}
    i = 1.0
    for key in dic: 
        if key not in keep_classes: mod_dic[key] = 0
        else: 
            mod_dic[key] = i
            if not binary: i +=1
    return mod_dic

In [215]:
keep_classes = [17.0, 53.0]
dc_mod = classes_to_keep(dc, keep_classes, binary = True)
dc_mod

{0.0: 0,
 2.0: 0,
 3.0: 0,
 4.0: 0,
 5.0: 0,
 7.0: 0,
 8.0: 0,
 10.0: 0,
 11.0: 0,
 12.0: 0,
 13.0: 0,
 14.0: 0,
 15.0: 0,
 16.0: 0,
 17.0: 1.0,
 18.0: 0,
 24.0: 0,
 26.0: 0,
 28.0: 0,
 30.0: 0,
 31.0: 0,
 41.0: 0,
 42.0: 0,
 43.0: 0,
 44.0: 0,
 46.0: 0,
 47.0: 0,
 49.0: 0,
 50.0: 0,
 51.0: 0,
 52.0: 0,
 53.0: 1.0,
 54.0: 0,
 58.0: 0,
 60.0: 0,
 62.0: 0,
 63.0: 0,
 77.0: 0,
 85.0: 0,
 251.0: 0,
 252.0: 0,
 253.0: 0,
 254.0: 0,
 255.0: 0,
 72.0: 0,
 80.0: 0}

In [216]:
def label_masks(studies_path, dic):
    
    for path in studies_path:
        study_id = (path.split("/")[-1])
        imgfn = (path + "/"+ study_id +"_seg.nii.gz")
        
        outfn = path + "/"+ study_id + "_m.nii.gz"
    
        img, rnx, rny, rnz, data, affine = load_nifti(imgfn)
            
        new_data = data #image matrix
        unique_labels = np.unique(data) #unique labels for the matrix
        for i in unique_labels:
            new_data[new_data == i] = (dic[i])
        new_data = new_data.astype('uint8')
        
        write_nifti(new_data, affine, outfn)

In [217]:
label_masks(studies_path, dc_mod)

## Remove images with depth below 150

In [170]:
def remove_folders(studies_path, depth_size):
 
    for path in studies_path:

        study_id = (path.split("/")[-1])
        imgfn = (path + "/"+ study_id +"_m.nii.gz")
        
        img, rnx, rny, rnz, data, affine = load_nifti(imgfn)
        depth = data.shape[2]
        if depth < depth_size: 
            shutil.rmtree(path)

In [171]:
depth_size = 150
removed_values = remove_folders(studies_path, depth_size)

# Normalize images 

In [187]:
def standardize_img(in_img):
    '''
    Input: in_img  - numpy ndarray 
    Output: out_img  - numpy ndarray with zero mean and unit variance
    '''
    out_img = (in_img - np.mean(in_img)) / np.std(in_img)
    return out_img

In [188]:
def normalization(studies_path, img_type):
    for path in studies_path:
        study_id = (path.split("/")[-1])
        img = f'{path}/{study_id}_{img_type}.nii.gz'
        save_fn = f'{path}/{study_id}_{img_type}std.nii.gz'
        
        if Path(save_fn).is_file():
            print(f"{out_fn} and the rest already processed?")
            break
        else:
            rfunc, rnx, rny, rnz, data, affine = load_nifti(img)
            out_data = standardize_img(data)
            write_nifti(out_data, affine, save_fn)

In [189]:
normalization(studies_path, 't2')